In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from cel.datasets import HelocDataset
from cel.discriminative_models import MultilayerPerceptron

In [4]:
dataset = HelocDataset("../data/heloc.csv")

In [5]:
disc_model = MultilayerPerceptron(dataset.X_test.shape[1], [512, 512], 2)
disc_model.load("heloc_disc_model.pt")

/home/lukasz/genwro/counterfactuals/counterfactuals/discriminative_models/multilayer_perceptron.py:109: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(to

In [6]:
disc_model

MultilayerPerceptron(
  (layers): ModuleList(
    (0): Linear(in_features=23, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=2, bias=True)
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (final_activation): Softmax(dim=1)
  (criterion): CrossEntropyLoss()
)

In [21]:
dataset.y_train = disc_model.predict(dataset.X_train).detach().numpy()
dataset.y_test = disc_model.predict(dataset.X_test).detach().numpy()

In [24]:
from cel.cf_methods.glance import GlobalGLANCE

global_glance = GlobalGLANCE(dataset, disc_model, k=30, m=1, s=4)
global_glance.prep()
X_cfs = global_glance.explain()
print("Validity: ")
disc_model.predict(X_cfs).sum() / X_cfs.shape[0]

100%|██████████| 1/1 [00:00<00:00,  4.70it/s]


Validity: 


tensor(0.9416)